In [ ]:
from models_impute import *
from pygrinder import block_missing

In [ ]:
data = pd.read_csv("./data/lorenz/lorenz_dataset_0_timeseries.csv",header=None)
data = data.values

In [ ]:
cg = pd.read_csv('./causality_matrices/lorenz_causality_matrix.csv', header=None)
cg = cg.values
model_params = {
        'num_levels': 10,
        'kernel_size': 8,
        'dilation_c': 2
    }

In [ ]:
data = block_missing(data[np.newaxis,...],factor=0.1, block_len=3, block_width=3)
data = data[0]
data

In [ ]:
data_imputed = impute(data, cg, model_params)
data_imputed

In [ ]:
# test_miracle.py
import numpy as np
from baseline import miracle_impu

# 创建测试数据
test_data = np.random.randn(100, 10)
test_data[np.random.random((100, 10)) < 0.1] = np.nan

print("测试数据形状:", test_data.shape)
print("缺失值数量:", np.isnan(test_data).sum())

try:
    result = miracle_impu(test_data)
    print("MIRACLE结果形状:", result.shape if result is not None else "None")
    print("MIRACLE结果类型:", type(result))
    
    if result is not None:
        print("填补后缺失值:", np.isnan(result).sum())
    else:
        print("❌ MIRACLE返回了None")
        
except Exception as e:
    print(f"❌ MIRACLE测试失败: {e}")
    import traceback
    traceback.print_exc()

In [15]:
import pandas as pd
import numpy as np

# 读取 CSV 文件
zero_matrix = pd.read_csv("zero_impu_matrix.csv").values  # shape: (T, N)

# 构造掩码矩阵：0 的位置为 0，其余为 1
mask_matrix = np.where(zero_matrix == 0, 0, 1)

# 保存新矩阵到 CSV
pd.DataFrame(mask_matrix).to_csv("zero_impu_mask.csv", index=False)


In [1]:
import numpy as np
import pandas as pd

In [7]:
pred = pd.read_csv('./train_result.csv').values
gt = pd.read_csv('./gt_matrix.csv').values

In [11]:
import torch.nn.functional as F
import torch

In [ ]:
res = F.mse_loss(torch.tensor(pred*mask_matrix), torch.tensor(gt*mask_matrix))
res

tensor(1398.8409, dtype=torch.float64)